# Age Prediction Model - Google Colab Version

This notebook trains age classification models (MobileNetV2, ResNet50, EfficientNet) on the Faces Age Detection Dataset.

## Setup Instructions
1. Go to Runtime > Change runtime type > Select **T4 GPU**
2. Run the cells below in order

## Step 1: Setup Kaggle API

To download the dataset, you need your Kaggle API credentials:
1. Go to https://www.kaggle.com/settings
2. Scroll to "API" section and click "Create New Token"
3. This downloads `kaggle.json` - upload it when prompted below

In [ ]:
# Upload your kaggle.json file
from google.colab import files
import os

# Create kaggle directory
os.makedirs('/root/.kaggle', exist_ok=True)

# Upload kaggle.json
print("Please upload your kaggle.json file:")
uploaded = files.upload()

# Move to correct location
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
print("Kaggle API configured!")

In [ ]:
# Download the dataset from Kaggle
!kaggle datasets download -d arashnic/faces-age-detection-dataset
!unzip -q faces-age-detection-dataset.zip -d /content/dataset
!rm faces-age-detection-dataset.zip
print("Dataset downloaded and extracted!")

In [ ]:
# Check dataset structure
!ls -la /content/dataset
!echo "---"
!ls -la /content/dataset/*

## Step 2: Imports and Setup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image
import os
import glob

In [ ]:
# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 3: Dataset Loading

This dataset has two parts. We'll use the UTKFace portion which has age in the filename.

In [ ]:
# Data directory - adjust based on the actual structure after extraction
# The UTKFace images should have filenames like: age_gender_race_datetime.jpg
DATA_DIR = "/content/dataset"

class AgeDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.files = []
        self.labels = []

        # Define age groups
        def categorize_age(age):
            if 0 <= age <= 25: return 0
            elif 26 <= age <= 50: return 1
            elif 51 <= age <= 75: return 2
            elif 76 <= age <= 116: return 3
            return -1

        # Find all image files recursively
        image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']
        all_images = []
        for ext in image_extensions:
            all_images.extend(glob.glob(os.path.join(data_dir, '**', ext), recursive=True))
        
        print(f"Found {len(all_images)} total images")
        
        # Process each image
        for img_path in all_images:
            filename = os.path.basename(img_path)
            if filename.startswith('.'): 
                continue
            
            try:
                # Try to extract age from filename (format: age_gender_race_datetime.jpg)
                age = int(filename.split('_')[0])
                grouped_label = categorize_age(age)
                if grouped_label != -1:
                    self.files.append(img_path)
                    self.labels.append(grouped_label)
            except (ValueError, IndexError):
                # Skip files that don't match the expected format
                continue
        
        print(f"Loaded {len(self.files)} valid images with age labels")
        
        # Print class distribution
        from collections import Counter
        dist = Counter(self.labels)
        print(f"Class distribution: {dict(sorted(dist.items()))}")
        print("  0: Age 0-25, 1: Age 26-50, 2: Age 51-75, 3: Age 76-116")

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_path = self.files[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [ ]:
# Define transformations
transforms_dict = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

# Load dataset and split
full_dataset = AgeDataset(DATA_DIR, transform=transforms_dict["train"])
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

print(f"\nDataset splits: Train={train_size}, Val={val_size}, Test={test_size}")

In [ ]:
# Define data loaders
dataloaders = {
    "train": DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2),
    "val": DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2),
    "test": DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2),
}
print("DataLoaders created!")

## Step 4: Model Definitions

In [ ]:
def get_model(model_name):
    if model_name == "mobilenet_v2":
        model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
        num_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 4)
        )
    
    elif model_name == "resnet50":
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        num_features = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 4)
        )

    elif model_name == "efficientnet_b0":
        model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        num_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 4)
        )
    
    else:
        raise ValueError("Invalid model name!")

    # Freeze feature extractor
    for param in model.parameters():
        param.requires_grad = False

    # Train new classifier layers
    if "efficientnet" in model_name or "mobilenet" in model_name:
        for param in model.classifier.parameters():
            param.requires_grad = True
    else:
        for param in model.fc.parameters():
            param.requires_grad = True
    
    return model.to(device)

## Step 5: Training and Testing Functions

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, model_name, num_epochs=10):
    best_acc = 0.0
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), f"best_{model_name}.pth")

    print(f"Best val Acc: {best_acc:.4f}")
    return model

In [ ]:
def test_model(model, dataloader, model_name):
    model.load_state_dict(torch.load(f"best_{model_name}.pth", weights_only=True))
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels).item()
            total += labels.size(0)
    print(f"Test Accuracy for {model_name}: {100 * correct / total:.2f}%")

In [ ]:
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(model, dataloader, model_name):
    model.load_state_dict(torch.load(f"best_{model_name}.pth", weights_only=True))
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['0-25', '26-50', '51-75', '76-116'], 
                yticklabels=['0-25', '26-50', '51-75', '76-116'])
    plt.xlabel("Predicted Age Group")
    plt.ylabel("True Age Group")
    plt.title(f"Confusion Matrix for {model_name}")
    plt.show()

## Step 6: Train Models

In [ ]:
# Train MobileNetV2
model_name = "mobilenet_v2"
print(f"\n{'='*50}")
print(f"Training {model_name}...")
print(f"{'='*50}\n")

model = get_model(model_name)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

model = train_model(model, dataloaders, criterion, optimizer, model_name, num_epochs=10)
test_model(model, dataloaders["test"], model_name)
plot_confusion_matrix(model, dataloaders["test"], model_name)

In [ ]:
# Train ResNet50
model_name = "resnet50"
print(f"\n{'='*50}")
print(f"Training {model_name}...")
print(f"{'='*50}\n")

model = get_model(model_name)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

model = train_model(model, dataloaders, criterion, optimizer, model_name, num_epochs=10)
test_model(model, dataloaders["test"], model_name)
plot_confusion_matrix(model, dataloaders["test"], model_name)

In [ ]:
# Train EfficientNet-B0
model_name = "efficientnet_b0"
print(f"\n{'='*50}")
print(f"Training {model_name}...")
print(f"{'='*50}\n")

model = get_model(model_name)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

model = train_model(model, dataloaders, criterion, optimizer, model_name, num_epochs=10)
test_model(model, dataloaders["test"], model_name)
plot_confusion_matrix(model, dataloaders["test"], model_name)

## Step 7: Grad-CAM Visualization

In [ ]:
import cv2

def denormalize_image(image_tensor):
    """Convert a normalized tensor image back to its original form for visualization"""
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    image = image_tensor.cpu().detach().numpy().transpose(1, 2, 0)
    image = (image * std) + mean
    image = np.clip(image, 0, 1)
    return (image * 255).astype(np.uint8)

def grad_cam(model, image_tensor, target_layer, class_idx=None):
    model.eval()
    gradients = None
    activations = None
    
    def backward_hook(module, grad_input, grad_output):
        nonlocal gradients
        gradients = grad_output[0]
    
    def forward_hook(module, input, output):
        nonlocal activations
        activations = output
    
    # Register hooks
    forward_handle = target_layer.register_forward_hook(forward_hook)
    backward_handle = target_layer.register_full_backward_hook(backward_hook)
    
    # Ensure image_tensor requires gradient
    image_tensor = image_tensor.unsqueeze(0).to(device).requires_grad_(True)
    
    # Forward pass
    output = model(image_tensor)
    
    if class_idx is None:
        class_idx = output.argmax().item()
    
    # Backpropagate
    model.zero_grad()
    loss = output[0, class_idx]
    loss.backward()
    
    # Remove hooks
    forward_handle.remove()
    backward_handle.remove()
    
    if gradients is None:
        raise ValueError("Gradients were not captured. Check if target_layer is correct.")
    
    # Compute Grad-CAM
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    for i in range(activations.shape[1]):
        activations[:, i, :, :] *= pooled_gradients[i]
    
    heatmap = torch.mean(activations, dim=1).squeeze().cpu().detach().numpy()
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8
    
    # Convert heatmap to color
    heatmap = cv2.resize(heatmap, (224, 224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    # Convert original image using denormalization
    original_image = denormalize_image(image_tensor.squeeze(0))
    
    # Blend heatmap with original image
    superimposed_img = cv2.addWeighted(original_image, 0.6, heatmap, 0.4, 0)
    
    # Display the image
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

    return superimposed_img

In [ ]:
def load_trained_model(model_name):
    model = get_model(model_name)
    model.load_state_dict(torch.load(f"best_{model_name}.pth", weights_only=True))
    model.eval()

    if model_name == "resnet50":
        target_layer = model.layer4[-1].conv3
    elif model_name == "mobilenet_v2":
        target_layer = model.features[-1][0]
    elif model_name == "efficientnet_b0":
        target_layer = model.features[-1][0]
    else:
        raise ValueError("Invalid model name!")

    return model, target_layer

def apply_grad_cam_to_images(model, target_layer, num_images=10):
    sample_images = [test_dataset[i][0] for i in range(num_images)]
    for i, image in enumerate(sample_images):
        print(f"Grad-CAM for Test Image {i+1}")
        grad_cam(model, image, target_layer)

In [ ]:
# Run Grad-CAM for each model
for model_name in ["mobilenet_v2", "resnet50", "efficientnet_b0"]:
    print(f"\n{'='*50}")
    print(f"Applying Grad-CAM for {model_name}...")
    print(f"{'='*50}\n")
    model, target_layer = load_trained_model(model_name)
    apply_grad_cam_to_images(model, target_layer, num_images=10)